In [78]:
# utils/llm.py

import os

from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_pinecone import PineconeVectorStore

text = '''
    넌 질문-답변을 도와주는 AI 영화 추천기야.
    아래 제공되는 Context를 통해서 사용자 Question에 대해 답을 해줘야해.

    Context에는 직접적으로 없어도, 추론하거나 계산할 수 있는 답변은 최대한 만들어 봐.

    답은 적절히 \n를 통해 문단을 나눠줘 한국어로 만들어 줘. 
    # Question:
    {question}

    # Context:
    {context}


    # Answer:
    '''
    
vectorstore = PineconeVectorStore.from_existing_index(
    index_name = os.environ.get('INDEX_NAME'),
    embedding=OpenAIEmbeddings()
)

# 5. Retrieve
retriever = vectorstore.as_retriever()


# 6. Prompting
prompt = PromptTemplate.from_template(text)
    
def query_llm(user_input):
    # 7. LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    parser = StrOutputParser()

    # 8. Chain
    chain = (
        {'context': retriever, 'question': RunnablePassthrough()}
        | prompt
        | llm
        | parser
    )

    ans = chain.invoke(user_input)
    return ans

In [79]:
from pydantic import BaseModel, Field

In [80]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

In [81]:
# 테스트용 코드
retrieval_grader = grade_prompt | structured_llm_grader
question = "80년대 배경의 전두환 등장 영화는?"  # Yes
docs = retriever.invoke(question)

In [82]:
print(docs)

[Document(id='dda6abfe-278f-44c9-bada-7fd4412b8bcb', metadata={'row': 29.0, 'source': './database.csv'}, page_content='id: 30\nhistory year: 1980\nhistory description: 80년대, 5.18민주화운동, 계엄군, 전두환, 광주, 학살\ngenre: \nmovie description: 김상경, 이요원\n저항, 슬픔, 비극\nmovie title: 화려한 휴가\nrelease date: 2007.07.25'), Document(id='3ab2cc02-2a9b-47bd-be06-98ae988d8595', metadata={'row': 29.0, 'source': './database.csv'}, page_content='id: 30\nhistory year: 1980\nhistory description: 80년대, 5.18민주화운동, 계엄군, 전두환, 광주, 학살\ngenre: \nmovie description: 김상경, 이요원\n저항, 슬픔, 비극\nmovie title: 화려한 휴가\nrelease date: 2007.07.25'), Document(id='9322a0df-5359-4da7-a40e-2d80ffb5fd2a', metadata={'row': 19.0, 'source': './database.csv'}, page_content='id: 20\nhistory year: 1964\nhistory description: 1960년 3.15 부정선거, 4.19혁명\n1961년 5.16군사정변\n1968년 김신조 사건\n60년대\ngenre: \nmovie description: 송강호, 대통령(독재자)의 이발사\n역사 속의 개인\nmovie title: 효자동 이발사\nrelease date: 2004.05.05'), Document(id='47bacd9e-ff93-4191-bed8-0d48d691c3e6', metadata=

In [83]:
print(docs[2])

page_content='id: 20
history year: 1964
history description: 1960년 3.15 부정선거, 4.19혁명
1961년 5.16군사정변
1968년 김신조 사건
60년대
genre: 
movie description: 송강호, 대통령(독재자)의 이발사
역사 속의 개인
movie title: 효자동 이발사
release date: 2004.05.05' metadata={'row': 19.0, 'source': './database.csv'}


In [84]:
doc_txt = docs[2].page_content

In [85]:
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [86]:
print(retrieval_grader.invoke({"question": question, "document": docs}))

# 241017 16:03

binary_score='yes'


In [87]:
### Generate

from langchain import hub # 좋은 프롬프트들을 가져와서 뜨게
from langchain_core.output_parsers import StrOutputParser

# Prompt
# prompt = hub.pull("rlm/rag-prompt") # 이게 가져오는 프롬프트 이름임

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

80년대 배경의 전두환이 등장하는 영화로는 **"화려한 휴가"**가 있습니다. 

이 영화는 2007년에 개봉되었으며, 5.18 민주화운동을 다루고 있습니다. 전두환과 관련된 역사적 사건들을 배경으로 하여, 저항과 슬픔, 비극을 주제로 하고 있습니다. 

주연으로는 김상경과 이요원이 출연하며, 이 영화는 80년대의 아픈 역사를 되새기게 하는 작품입니다.


In [88]:
### Question Re-writer

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question = "야구 역대 최고 관중 수는 얼마인가요?" # Yes

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'역대 야구 경기 중 최고 관중 수는 얼마인가요?'

In [89]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=3)

web_search_tool.invoke('정마담이 빌린 돈은?')

# 24-10-21 11:12

[{'url': 'https://www.kmib.co.kr/article/view.asp?arcid=0013476407',
  'content': "정마담이 폭로 가담한 이유 입력: 2019-07-09 11:24. 수정: ... (돈을 빌린 사람)들이 금리 인하를 체감하기까지 상당한 시간이 필요할 전망이다. 시장이 기준금리 인하 기대감을 선반영한 데다 금융 당 금리는 낮췄지만 정부 재정 '바닥'… 내수회복에 쓸 예산이 없다"},
 {'url': 'https://www.etoday.co.kr/news/view/1760604',
  'content': '정마담, \'연예계 포주\' 의혹 받는 그 정체는?…기획사와 친분 깊다는 이야기도 정마담 누구길래 정마담, 연예 기획사와 친분도? \'연예계 포주\' 의혹을 받고 있는 정마담의 정체에 대중의 시선이 집중되고 있다. 특히 ㄱ씨는 "식사에 참석한 여성 중 일부가 \'정마담\'이라 불리는 유흥업소 관계자를 통해 동원됐다"라고 주장했다. 이에 \'정마담\'이 누구인지 많은 이들이 궁금해하고 있는 상황. 다만 제보자의 주장에 따르면 정마담은 YG 엔터테인먼트 측과 깊은 친분을 유지하고 있는 것으로 전해졌다. 유흥업소 관계자라는 정마담이 연예계까지 손을 뻗고 있는 것. 관련 뉴스 주요 뉴스 많이 본 뉴스 문화·라이프 최신 뉴스 마켓 뉴스 (주)이투데이 (제호 : 이투데이) ㅣ 서울시 강남구 강남대로 556 이투데이빌딩 ㅣ ☎ 02) 799-2600 ㅣ 보도자료 및 기사제보 press@etoday.co.kr 이투데이 임직원은 모두의 의견을 모아 언론 윤리강령, 기자윤리강령, 임직원 윤리강령 및 실천규정을 제정, 준수하고 있습니다. 한국기자협회와 인터넷신문위원회 윤리강령 및 실천요강도 준수합니다.'},
 {'url': 'https://www.mk.co.kr/news/hot-issues/8870477',
  'content': '뉴스 바로가기 뉴스 [매일경제 스타투데이 차윤주 인턴기자] \'정마담\'이 조 로우, 양현석, 싸이, 

In [90]:
from typing import List

from typing_extensions import TypedDict

# 사전 세팅 단계
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

In [99]:
from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents from the CSV context.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE FROM CSV---")
    question = state["question"]

    # CSV에서 문서 검색
    docs = retriever.invoke(question)
    
    # 검색된 문서에 source 정보를 추가하여 반환
    documents = [Document(page_content=doc.page_content, metadata={"source": "CSV"}) for doc in docs]

    return {"documents": documents, "question": question, "source": "CSV"}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results and source as 'web'
    """
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])

    # Web search 실행
    docs = web_search_tool.invoke({"query": question})
    
    # 웹 검색 결과를 새로운 문서로 추가 (source 정보를 각 문서의 metadata에 추가)
    for d in docs:
        web_result = Document(page_content=d["content"], metadata={"source": "web"})
        documents.append(web_result)

    return {"documents": documents, "question": question, "source": "web"}




### Edges

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call ('generate_answer' or 'transform_query')
    """
    print("---ASSESS GRADED DOCUMENTS---")
    documents = state["documents"]

    # 문서가 있는지 여부에 따라 결정
    if documents:
        # 문서가 있으면 답변을 생성
        print("---DECISION: GENERATE ANSWER---")
        return "generate_answer"  # 문서가 있으면 바로 답변 생성
    else:
        # 문서가 없으면 질문을 변환하여 웹 검색
        print("---DECISION: TRANSFORM QUERY AND SEARCH---")
        return "transform_query"  # 문서가 없으면 질문 변환으로 이동


def generate_answer(state):
    """
    Generate answer based on whether relevant movie exists in the context.
    """
    print("---GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]

    # 문서가 없는 경우 먼저 처리
    if not documents:
        return {
            "documents": documents,
            "question": question,
            "generation": "죄송합니다. 관련 영화가 없습니다. 다른 영화를 추천 받으시겠어요?"
        }

    # 기본 답변 생성
    generation = rag_chain.invoke({"context": documents, "question": question})
    
    # 문서 소스 확인 및 메시지 추가
    has_csv = any(doc.metadata.get("source") == "CSV" for doc in documents)
    has_web = any(doc.metadata.get("source") == "web" for doc in documents)
    
    # 조건에 따라 적절한 메시지 추가
    if has_csv:
        generation += "\n해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다."
    elif has_web:
        generation += "\nPOV Timeline은 해당 정보를 갖고 있지 않아, 관련 영화를 말씀드렸습니다. 해당 영화가 궁금하시다면 웹 검색을 추천드립니다."

    return {
        "documents": documents,
        "question": question,
        "generation": generation
    }

In [100]:
from langgraph.graph import END, StateGraph, START

# 5. 워크플로우 설정
workflow = StateGraph(GraphState)

# 각 노드 정의
workflow.add_node("retrieve", retrieve)  # CSV 검색 노드
workflow.add_node("grade_documents", grade_documents)  # 문서 평가 노드
workflow.add_node("generate_answer", generate_answer)  # 답변 생성 노드
workflow.add_node("transform_query", transform_query)  # 질문 변환 노드
workflow.add_node("web_search_node", web_search)  # 웹 검색 노드

# decide_to_generate 함수 정의
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.
    """
    print("---ASSESS GRADED DOCUMENTS---")
    documents = state["documents"]
    
    if documents:
        # 문서가 있으면 답변을 생성
        print("---DECISION: GENERATE ANSWER---")
        return "generate_answer"  
    else:
        # 문서가 없으면 질문 변환으로 이동
        print("---DECISION: TRANSFORM QUERY AND SEARCH---")
        return "transform_query"

# 엣지 설정
workflow.add_edge(START, "retrieve")  # 시작: CSV에서 검색
workflow.add_edge("retrieve", "grade_documents")  # 문서 평가로 이동

# 문서 평가 후 조건에 따라 질문 변환 또는 답변 생성
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,  # 문서가 적합한지 평가하여
    {
        "transform_query": "transform_query",  # 문서가 없으면 질문 변환
        "generate_answer": "generate_answer",  # 문서가 있으면 답변 생성
    },
)

# 질문 변환 후 웹 검색으로 이어지도록 설정
workflow.add_edge("transform_query", "web_search_node")  # 변환된 질문으로 웹 검색
workflow.add_edge("web_search_node", "generate_answer")  # 웹 검색 후 답변 생성

# 답변 생성 후 종료
workflow.add_edge("generate_answer", END)  # 답변 생성 후 종료

# Compile
app = workflow.compile()


In [102]:
from pprint import pprint

# Run
inputs = {"question": "윤동주 관련 영화 알려줘"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE FROM CSV---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE ANSWER---
"Node 'grade_documents':"
'\n---\n'
---GENERATE ANSWER---
"Node 'generate_answer':"
'\n---\n'
('윤동주와 관련된 영화로는 **"동주"**가 있습니다. \n'
 '\n'
 '이 영화는 2016년 2월 17일에 개봉하였으며, 이준익 감독이 메가폰을 잡았습니다. 주연으로는 박정민과 강하늘이 출연하여 윤동주와 그의 '
 '친구 송몽규의 이야기를 담고 있습니다. \n'
 '\n'
 '영화는 흑백으로 제작되었으며, 윤동주라는 민족시인의 비극적이고 슬픈 삶을 조명합니다. 일제 강점기라는 역사적 배경 속에서 독립을 향한 '
 '저항과 민족의 아픔을 표현하고 있습니다. \n'
 '\n'
 '윤동주에 대한 깊은 이해와 감동을 주는 이 작품을 추천드립니다.\n'
 '해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다.')


In [103]:
# Run
inputs = {"question": "전태일 열사가 등장하는 다룬 한국 영화 추천해줘"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE FROM CSV---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: TRANSFORM QUERY AND SEARCH---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search_node':"
'\n---\n'
---GENERATE ANSWER---
"Node 'generate_answer':"
'\n---\n'
('전태일 열사가 주인공으로 등장하는 한국 영화로는 다음과 같은 작품들이 있습니다.\n'
 '\n'
 '첫 번째로, **"태일이"**라는 영화가 있습니다. 이 영화는 전태일 열사의 삶과 투쟁을 다룬 작품으로, 그의 고난과 희생을 통해 '
 '노동자의 권리와 인권 문제를 조명합니다. \n'
 '\n'
 '또한, 전태일 열사의 이야기를 다룬 다큐멘터리 형식의 영화들도 존재합니다. 이러한 작품들은 그의 생애와 시대적 배경을 깊이 있게 '
 '탐구하며, 관객들에게 감동적인 메시지를 전달합니다.\n'
 '\n'
 '이 외에도 전태일 열사와 관련된 다양한 작품들이 있으니, 관심이 있다면 추가적인 자료를 찾아보시는 것도 좋습니다.\n'
 'POV Timeline은 해당 정보를 갖고 있지 않아, 관련 영화를 말씀드렸습니다. 해당 영화가 궁금하시다면 웹 검색을 추천드립니다.')
